# Lab 10 NN, ReLu, Xavier, Dropout, and Adam

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices()

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 1. Softmax classifier for MNIST

In [2]:
# Lab 7 Learning rate and Evaluation
import tensorflow as tf

learning_rate = 0.001
batch_size = 100
training_epochs = 30
nb_classes = 10

mnist = tf.keras.datasets.mnist

# load_data mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("x_train\n")
print(x_train)
print("x_test\n")
print(x_test)

# normalizing data
x_train, x_test = x_train / 255.0, x_test / 255.0

print("==========================================================")

print("x_train\n")
print(x_train)
print("x_test\n")
print(x_test)

x_train

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
x_test

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 

In [3]:
print(x_train.shape)  # (60000, 28, 28)
print(x_test.shape)


# change data shape
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])


print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000, 784)
(10000, 784)


In [4]:
print(y_test.shape)
print(y_test)

# change result to one-hot encoding
# in tf1, one_hot= True in read_data_sets("MNIST_data/", one_hot=True)
# took care of it, but here we need to manually convert them
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

print(y_test.shape)
print(y_test)

# # Consider an array of 5 labels out of a set of 3 classes {0, 1, 2}:
# array([0, 2, 1, 2, 0])
# `to_categorical` converts this into a matrix with as many columns as there are classes. The number of rows
#  stays the same. to_categorical(labels)
# array([[ 1.,  0.,  0.],
#        [ 0.,  0.,  1.],
#        [ 0.,  1.,  0.],
#        [ 0.,  0.,  1.],
#        [ 1.,  0.,  0.]], dtype=float32)

(10000,)
[7 2 1 ... 4 5 6]
(10000, 10)
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
# Sequential Model
# keras의 sequential model은 정의한 다음 아래의 예제 코드처럼 차례로 계층(layer)을 쌓아 나가면 되는 매우 간단한 구조이다.
tf.model = tf.keras.Sequential()

tf.model.add(tf.keras.layers.Dense(units=10, input_dim=784, activation='softmax'))

# Compilation
# Sequential 모델을 정의하고 학습하기 전에 .compile() 메서드를 사용하여 학습에 대한 설정을 해줘야한다.
#   Optimizer: 최적화 함수를 설정하는 부분이며, 'sgd', 'adam', 'rmsprop' 등 문자열타입으로 설정할 수 있다.
#   Loss function: 손실함수를 설정해주는 부분이며, 'categorical_crossentropy', 'mse' 처럼 문자열타입으로 설정할 수 있다.
#   Metrics: 모델의 성능을 판정하는데 사용하는 지표 함수이며,['accuracy'] 처럼 리스트 형태로 설정한다.
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.Adam(0.001), metrics=['accuracy'])


print("***************************************************************************************************")
tf.model.summary()
print("***************************************************************************************************\n\n")


# learning_rate = 0.001
# batch_size = 100
# training_epochs = 30
# nb_classes = 10


# Training
# 마지막으로 학습을 하기위해 .fit()에 학습할 데이터를 넣어주고, 에폭(epochs)과 배치크기(batch_size)를 설정해준다.
history = tf.model.fit(x_train, y_train, batch_size=batch_size, epochs=training_epochs)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

***************************************************************************************************
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                7850      
                                                                 
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________
***************************************************************************************************




2022-08-26 16:17:32.912684: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-26 16:17:32.912820: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/30


2022-08-26 16:17:33.259610: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-26 16:17:33.404721: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


600/600 [==============================] - 3s 5ms/step - loss: 0.6270 - accuracy: 0.8427
Epoch 2/30
600/600 [==============================] - 3s 6ms/step - loss: 0.3467 - accuracy: 0.9051
Epoch 3/30
600/600 [==============================] - 3s 5ms/step - loss: 0.3096 - accuracy: 0.9147
Epoch 4/30
600/600 [==============================] - 4s 7ms/step - loss: 0.2924 - accuracy: 0.9185
Epoch 5/30
600/600 [==============================] - 4s 6ms/step - loss: 0.2821 - accuracy: 0.9217
Epoch 6/30
600/600 [==============================] - 3s 6ms/step - loss: 0.2748 - accuracy: 0.9231
Epoch 7/30
600/600 [==============================] - 3s 6ms/step - loss: 0.2698 - accuracy: 0.9245
Epoch 8/30
600/600 [==============================] - 4s 6ms/step - loss: 0.2657 - accuracy: 0.9258
Epoch 9/30
600/600 [==============================] - 3s 5ms/step - loss: 0.2623 - accuracy: 0.9262
Epoch 10/30
600/600 [==============================] - 3s 6ms/step - loss: 0.2593 - accuracy: 0.9284
Epoch 11/3

In [6]:
# evaluate test set
predictions = tf.model.predict(x_test)
print('Prediction: \n', predictions)
x_train

# 'output_b': ['mse', 'accuracy']
score = tf.model.evaluate(x_test, y_test)
print('Accuracy: ', score[1])

135/313 [===========>..................] - ETA: 0s

2022-08-26 16:19:15.398541: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 0s 1ms/step
Prediction: 
 [[1.01764222e-07 1.49107777e-13 3.78530672e-07 ... 9.95813608e-01
  8.85491590e-06 1.78110684e-04]
 [4.97783913e-05 1.91096410e-06 9.92091775e-01 ... 5.65881568e-21
  1.03371567e-05 1.14358654e-17]
 [3.91083461e-07 9.87801135e-01 7.45791662e-03 ... 7.39117022e-05
  1.63689279e-03 1.27808555e-04]
 ...
 [3.88562382e-09 1.07516784e-09 2.00762815e-06 ... 1.38462870e-03
  8.63794424e-03 1.32458992e-02]
 [9.31311384e-09 3.43651720e-08 1.44064867e-08 ... 4.72502677e-08
  5.26227010e-03 1.38682976e-08]
 [5.47860246e-08 1.45667739e-15 2.37531567e-05 ... 1.01709387e-16
  2.29726460e-09 3.48661936e-13]]
 31/313 [=>............................] - ETA: 1s - loss: 0.2764 - accuracy: 0.9244

2022-08-26 16:19:15.888912: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 5ms/step - loss: 0.2669 - accuracy: 0.9271
Accuracy:  0.9271000623703003


# 2. NN for MNIST ( ReLU )

    * Output Layer 에서는 출력을 0과 1 사이여야하므로 sigmoid function을 이용해야한다.

In [7]:
import numpy as np
import random
import tensorflow as tf

random.seed(777)  # for reproducibility
learning_rate = 0.001
batch_size = 100
training_epochs = 30
nb_classes = 10

# load_data mnist
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


print(x_train.shape)  # (60000, 28, 28)


# change data shape
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])

# one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

(60000, 28, 28)


In [8]:
# Sequential Model
tf.model = tf.keras.Sequential()


tf.model.add(tf.keras.layers.Dense(input_dim=784, units=256, activation='relu'))
tf.model.add(tf.keras.layers.Dense(units=256, activation='relu'))
tf.model.add(tf.keras.layers.Dense(units=nb_classes, activation='softmax'))


# Compilation
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

# summary
tf.model.summary()

# Training
tf.model.fit(x_train, y_train, batch_size=batch_size, epochs=training_epochs)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 256)               200960    
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                                 
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30


/Users/mindongjun/miniconda3/envs/tf2/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-08-26 16:19:17.980296: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


600/600 [==============================] - 4s 6ms/step - loss: 2.1001 - accuracy: 0.8987
Epoch 2/30
600/600 [==============================] - 4s 6ms/step - loss: 0.4177 - accuracy: 0.9482
Epoch 3/30
600/600 [==============================] - 4s 6ms/step - loss: 0.2617 - accuracy: 0.9599
Epoch 4/30
600/600 [==============================] - 4s 6ms/step - loss: 0.2074 - accuracy: 0.9645
Epoch 5/30
600/600 [==============================] - 4s 6ms/step - loss: 0.1728 - accuracy: 0.9681
Epoch 6/30
600/600 [==============================] - 4s 6ms/step - loss: 0.1238 - accuracy: 0.9745
Epoch 7/30
600/600 [==============================] - 4s 6ms/step - loss: 0.1123 - accuracy: 0.9755
Epoch 8/30
600/600 [==============================] - 4s 6ms/step - loss: 0.0998 - accuracy: 0.9772
Epoch 9/30
600/600 [==============================] - 4s 6ms/step - loss: 0.0796 - accuracy: 0.9809
Epoch 10/30
600/600 [==============================] - 4s 6ms/step - loss: 0.0742 - accuracy: 0.9816
Epoch 11/3

In [9]:
# predict 10 random hand-writing data
y_predicted = tf.model.predict(x_test)
for x in range(0, 10):
    random_index = random.randint(0, x_test.shape[0]-1)
    print("index: ", random_index,
          "actual y: ", np.argmax(y_test[random_index]),
          "predicted y: ", np.argmax(y_predicted[random_index]))

# evaluate test set
# 'output_b': ['mse', 'accuracy']
evaluation = tf.model.evaluate(x_test, y_test)
print('loss: ', evaluation[0])
print('accuracy', evaluation[1])

 65/313 [=====>........................] - ETA: 0s

2022-08-26 16:21:12.010985: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 1s 2ms/step
index:  3757 actual y:  8 predicted y:  8
index:  7304 actual y:  5 predicted y:  5
index:  7300 actual y:  7 predicted y:  7
index:  6039 actual y:  9 predicted y:  9
index:  9429 actual y:  3 predicted y:  3
index:  4420 actual y:  5 predicted y:  5
index:  5507 actual y:  2 predicted y:  2
index:  8809 actual y:  1 predicted y:  1
index:  654 actual y:  5 predicted y:  8
index:  7302 actual y:  8 predicted y:  8
 27/313 [=>............................] - ETA: 1s - loss: 0.2251 - accuracy: 0.9688

2022-08-26 16:21:12.693010: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 6ms/step - loss: 0.2210 - accuracy: 0.9727
loss:  0.2210409790277481
accuracy 0.9727000594139099


# 3. Xavier for MNIST

> * 초기화를 잘 시키는 방법
> * The Glorot normal initializer, also called Xavier normal initializer.

    1. glorot_normal
        Normal Distribution
        " keras.initializers.glorot_normal(seed=None) "
        Glorot 정규분포 초기값 설정기, Xavier 정규분포 초기값 설정기라고 한다.

    2. glorot_uniform
        Uniform Distribution
        " keras.initializers.glorot_uniform(seed=None) "
        Glorot 균등분포 초기값 설정기, Xavier 균등분포 초기값 설정기라고 한다.

    3. HeNormal
        " tf.keras.initializers.HeNormal(seed=None) "

    4. HeUniform
        " tf.keras.initializers.HeUniform(seed=None) "

Activation function으로 ReLU를 사용할 때는 He’s Initialization을 사용하고 sigdmoid나 tanh 등의 S자 모양 곡선일 때는 Xavier Initinaliztion을 사용한다.

In [10]:
import numpy as np
import random
import tensorflow as tf

random.seed(777)  # for reproducibility
learning_rate = 0.001
batch_size = 100
training_epochs = 30
nb_classes = 10

# load_data mnist
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


print(x_train.shape)  # (60000, 28, 28)


# change data shape
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])

# one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

(60000, 28, 28)


In [11]:
# Sequential Model
tf.model = tf.keras.Sequential()
# Glorot normal initializer, also called Xavier normal initializer.
# see https://www.tensorflow.org/api_docs/python/tf/initializers


Xavier = tf.keras.initializers.glorot_normal()
He = tf.keras.initializers.he_normal()
tf.model.add(tf.keras.layers.Dense(input_dim=784, units=256, kernel_initializer=He, activation='relu'))
tf.model.add(tf.keras.layers.Dense(units=256, kernel_initializer=He, activation='relu'))
tf.model.add(tf.keras.layers.Dense(units=nb_classes, kernel_initializer=Xavier, activation='softmax'))

# Compilation
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

# summary
tf.model.summary()

# Training
history = tf.model.fit(x_train, y_train, batch_size=batch_size, epochs=training_epochs)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 256)               200960    
                                                                 
 dense_5 (Dense)             (None, 256)               65792     
                                                                 
 dense_6 (Dense)             (None, 10)                2570      
                                                                 
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
 16/600 [..............................] - ETA: 4s - loss: 44.5461 - accuracy: 0.5300  

2022-08-26 16:21:15.084988: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


600/600 [==============================] - 4s 6ms/step - loss: 3.6680 - accuracy: 0.8942
Epoch 2/30
600/600 [==============================] - 4s 6ms/step - loss: 0.7351 - accuracy: 0.9447
Epoch 3/30
600/600 [==============================] - 4s 6ms/step - loss: 0.4497 - accuracy: 0.9571
Epoch 4/30
600/600 [==============================] - 3s 6ms/step - loss: 0.3407 - accuracy: 0.9628
Epoch 5/30
600/600 [==============================] - 4s 6ms/step - loss: 0.2500 - accuracy: 0.9683
Epoch 6/30
600/600 [==============================] - 4s 6ms/step - loss: 0.2270 - accuracy: 0.9706
Epoch 7/30
600/600 [==============================] - 4s 6ms/step - loss: 0.1966 - accuracy: 0.9719
Epoch 8/30
600/600 [==============================] - 3s 6ms/step - loss: 0.1390 - accuracy: 0.9763
Epoch 9/30
600/600 [==============================] - 4s 6ms/step - loss: 0.1260 - accuracy: 0.9770
Epoch 10/30
600/600 [==============================] - 3s 6ms/step - loss: 0.1151 - accuracy: 0.9784
Epoch 11/3

In [12]:
# predict 10 random hand-writing data
y_predicted = tf.model.predict(x_test)
for x in range(0, 10):
    random_index = random.randint(0, x_test.shape[0]-1)
    print("index: ", random_index, "actual y: ", np.argmax(y_test[random_index]), "predicted y: ", np.argmax(y_predicted[random_index]))

# evaluate test set
# 'output_b': ['mse', 'accuracy']
evaluation = tf.model.evaluate(x_test, y_test)
print('loss: ', evaluation[0])
print('accuracy', evaluation[1])

108/313 [=========>....................] - ETA: 0s

2022-08-26 16:23:01.318781: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 1s 2ms/step
index:  3757 actual y:  8 predicted y:  3
index:  7304 actual y:  5 predicted y:  5
index:  7300 actual y:  7 predicted y:  7
index:  6039 actual y:  9 predicted y:  9
index:  9429 actual y:  3 predicted y:  3
index:  4420 actual y:  5 predicted y:  5
index:  5507 actual y:  2 predicted y:  2
index:  8809 actual y:  1 predicted y:  1
index:  654 actual y:  5 predicted y:  5
index:  7302 actual y:  8 predicted y:  8
 30/313 [=>............................] - ETA: 1s - loss: 0.2654 - accuracy: 0.9677

2022-08-26 16:23:01.907344: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 6ms/step - loss: 0.2282 - accuracy: 0.9702
loss:  0.22820210456848145
accuracy 0.9702000617980957


# 4. Deep NN for MNIST

In [13]:
# Lab 10 MNIST and Deep learning
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt

random.seed(777)  # for reproducibility
learning_rate = 0.001
batch_size = 100
training_epochs = 30
nb_classes = 10

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


print(x_train.shape)  # (60000, 28, 28)


# change data shape
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])


# one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

(60000, 28, 28)


In [14]:
# Sequential Model
tf.model = tf.keras.Sequential()
# Glorot normal initializer, also called Xavier normal initializer.
# see https://www.tensorflow.org/api_docs/python/tf/initializers

Xavier = tf.keras.initializers.glorot_normal()
He = tf.keras.initializers.he_normal()
tf.model.add(tf.keras.layers.Dense(input_dim=784, units=512, kernel_initializer=He, activation='relu'))
tf.model.add(tf.keras.layers.Dense(units=512, kernel_initializer=He, activation='relu'))
tf.model.add(tf.keras.layers.Dense(units=512, kernel_initializer=He, activation='relu'))
tf.model.add(tf.keras.layers.Dense(units=512, kernel_initializer=He, activation='relu'))
tf.model.add(tf.keras.layers.Dense(units=nb_classes, kernel_initializer=Xavier, activation='softmax'))

# Compilation
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

# summary
tf.model.summary()

# Training
history = tf.model.fit(x_train, y_train, batch_size=batch_size, epochs=training_epochs)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 512)               401920    
                                                                 
 dense_8 (Dense)             (None, 512)               262656    
                                                                 
 dense_9 (Dense)             (None, 512)               262656    
                                                                 
 dense_10 (Dense)            (None, 512)               262656    
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 1,195,018
Trainable params: 1,195,018
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30


/Users/mindongjun/miniconda3/envs/tf2/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


  2/600 [..............................] - ETA: 41s - loss: 185.7281 - accuracy: 0.1200 

2022-08-26 16:23:04.498152: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


600/600 [==============================] - 5s 8ms/step - loss: 3.3700 - accuracy: 0.8877
Epoch 2/30
600/600 [==============================] - 4s 7ms/step - loss: 0.2692 - accuracy: 0.9480
Epoch 3/30
600/600 [==============================] - 4s 7ms/step - loss: 0.1717 - accuracy: 0.9609
Epoch 4/30
600/600 [==============================] - 5s 8ms/step - loss: 0.1315 - accuracy: 0.9682
Epoch 5/30
600/600 [==============================] - 5s 8ms/step - loss: 0.1113 - accuracy: 0.9712
Epoch 6/30
600/600 [==============================] - 5s 8ms/step - loss: 0.0916 - accuracy: 0.9755
Epoch 7/30
600/600 [==============================] - 5s 8ms/step - loss: 0.0978 - accuracy: 0.9743
Epoch 8/30
600/600 [==============================] - 5s 8ms/step - loss: 0.0800 - accuracy: 0.9778
Epoch 9/30
600/600 [==============================] - 5s 8ms/step - loss: 0.0636 - accuracy: 0.9812
Epoch 10/30
600/600 [==============================] - 5s 8ms/step - loss: 0.0670 - accuracy: 0.9808
Epoch 11/3

In [15]:
# predict 10 random hand-writing data
y_predicted = tf.model.predict(x_test)
for x in range(0, 10):
    random_index = random.randint(0, x_test.shape[0]-1)
    print("index: ", random_index, "actual y: ", np.argmax(y_test[random_index]), "predicted y: ", np.argmax(y_predicted[random_index]))

# evaluate test set
# 'output_b': ['mse', 'accuracy']
evaluation = tf.model.evaluate(x_test, y_test)
print('loss: ', evaluation[0])
print('accuracy', evaluation[1])

 96/313 [========>.....................] - ETA: 0s

2022-08-26 16:25:34.866846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 1s 2ms/step
index:  3757 actual y:  8 predicted y:  8
index:  7304 actual y:  5 predicted y:  5
index:  7300 actual y:  7 predicted y:  7
index:  6039 actual y:  9 predicted y:  9
index:  9429 actual y:  3 predicted y:  3
index:  4420 actual y:  5 predicted y:  5
index:  5507 actual y:  2 predicted y:  2
index:  8809 actual y:  1 predicted y:  1
index:  654 actual y:  5 predicted y:  5
index:  7302 actual y:  8 predicted y:  8
 18/313 [>.............................] - ETA: 1s - loss: 0.1073 - accuracy: 0.9809

2022-08-26 16:25:35.530693: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 7ms/step - loss: 0.1291 - accuracy: 0.9794
loss:  0.12908074259757996
accuracy 0.9794000387191772


# 5. Dropout for MNIST

In [16]:
# Lab 10 MNIST and Deep learning
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt

random.seed(777)  # for reproducibility
learning_rate = 0.001
batch_size = 100
training_epochs = 30
nb_classes = 10

# load_data mnist
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


print(x_train.shape)  # (60000, 28, 28)


# change data shape
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])

# one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

(60000, 28, 28)


In [17]:
# Sequential Model
tf.model = tf.keras.Sequential()
# Glorot normal initializer, also called Xavier normal initializer.
# see https://www.tensorflow.org/api_docs/python/tf/initializers



# 0.7(70%)에 해당하는 노드들만 남겨두고 다 지워버리는 것
# Float between 0 and 1. Fraction of the input units to drop.
drop_rate = 0.3

Xavier = tf.keras.initializers.glorot_normal()
He = tf.keras.initializers.he_normal()

tf.model.add(tf.keras.layers.Dense(input_dim=784, units=512, kernel_initializer=He, activation='relu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))

tf.model.add(tf.keras.layers.Dense(units=512, kernel_initializer=He, activation='relu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))

tf.model.add(tf.keras.layers.Dense(units=512, kernel_initializer=He, activation='relu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))

tf.model.add(tf.keras.layers.Dense(units=512, kernel_initializer=He, activation='relu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))

tf.model.add(tf.keras.layers.Dense(units=nb_classes, kernel_initializer=Xavier, activation='softmax'))


# Compilation
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

# summary
tf.model.summary()

# Training
history = tf.model.fit(x_train, y_train, batch_size=batch_size, epochs=training_epochs)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_14 (Dense)            (None, 512)               262656    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 512)              

2022-08-26 16:25:38.191503: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


600/600 [==============================] - 7s 11ms/step - loss: 7.9183 - accuracy: 0.7425
Epoch 2/30
600/600 [==============================] - 6s 11ms/step - loss: 0.6701 - accuracy: 0.8511
Epoch 3/30
600/600 [==============================] - 7s 11ms/step - loss: 0.4004 - accuracy: 0.8948
Epoch 4/30
600/600 [==============================] - 6s 10ms/step - loss: 0.3013 - accuracy: 0.9184
Epoch 5/30
600/600 [==============================] - 6s 9ms/step - loss: 0.2538 - accuracy: 0.9308
Epoch 6/30
600/600 [==============================] - 5s 9ms/step - loss: 0.2216 - accuracy: 0.9400
Epoch 7/30
600/600 [==============================] - 5s 9ms/step - loss: 0.1986 - accuracy: 0.9440
Epoch 8/30
600/600 [==============================] - 5s 9ms/step - loss: 0.1897 - accuracy: 0.9473
Epoch 9/30
600/600 [==============================] - 5s 8ms/step - loss: 0.1788 - accuracy: 0.9520
Epoch 10/30
600/600 [==============================] - 5s 9ms/step - loss: 0.1757 - accuracy: 0.9528
Epoch 

In [18]:
# predict 10 random hand-writing data
y_predicted = tf.model.predict(x_test)
for x in range(0, 10):
    random_index = random.randint(0, x_test.shape[0]-1)
    print("index: ", random_index, "actual y: ", np.argmax(y_test[random_index]), "predicted y: ", np.argmax(y_predicted[random_index]))

# evaluate test set
# 'output_b': ['mse', 'accuracy']
evaluation = tf.model.evaluate(x_test, y_test)
print('loss: ', evaluation[0])
print('accuracy', evaluation[1])

 88/313 [=======>......................] - ETA: 0s

2022-08-26 16:28:25.663004: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 1s 2ms/step
index:  3757 actual y:  8 predicted y:  3
index:  7304 actual y:  5 predicted y:  5
index:  7300 actual y:  7 predicted y:  7
index:  6039 actual y:  9 predicted y:  9
index:  9429 actual y:  3 predicted y:  3
index:  4420 actual y:  5 predicted y:  5
index:  5507 actual y:  2 predicted y:  2
index:  8809 actual y:  1 predicted y:  1
index:  654 actual y:  5 predicted y:  5
index:  7302 actual y:  8 predicted y:  8
 19/313 [>.............................] - ETA: 1s - loss: 0.0823 - accuracy: 0.9770

2022-08-26 16:28:26.320604: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 6ms/step - loss: 0.1415 - accuracy: 0.9762
loss:  0.1415245234966278
accuracy 0.9762000441551208
